# RWC-Minus-Studio

RWC-Minus-Studio is a web application built with Flask that leverages Stable Diffusion models for image generation. It provides endpoints for text-to-image and image-to-image generation, as well as a simple frontend for interaction.

## Setup

1. **Install Dependencies:**
   Run the following command in your Colab notebook to install the required libraries:

In [17]:
!pip install --quiet diffusers transformers accelerate scipy xformers torchvision torch safetensors xformers huggingface_hub Flask flask-ngrok pillow flask-cors pydrive2 requests

from huggingface_hub import login
login(token="xxxxx") # hf token here

## ***Source Code***

In [ ]:
import os
import string
import random
import time
import base64
import traceback
from io import BytesIO
from typing import Dict, Any
from functools import wraps
from threading import Semaphore

from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from werkzeug.utils import secure_filename
from PIL import Image
import torch
import requests

from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline
from transformers import logging as transformers_logging

try:
    from google.colab import auth as colab_auth
except Exception:
    colab_auth = None

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

transformers_logging.set_verbosity_error()

# --------------- CONFIG ---------------
MODEL_ID = os.environ.get("MODEL_ID", "Lykon/DreamShaper")
HF_TOKEN = os.environ.get("HF_TOKEN")  # set if required
DEVICE = os.environ.get("DEVICE", "cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/drive/MyDrive/Result")
INPUT_DIR = os.environ.get("INPUT_DIR", "/content/drive/MyDrive/InputImages")
META_DIR = os.environ.get("META_DIR", "/content/drive/MyDrive/ResultMeta")
NGROK_DOMAIN = os.environ.get("NGROK_DOMAIN", "yourmastertrainai.pagekite.me")  # optional
API_KEY = os.environ.get("API_KEY", "changeme_local_only")
ALLOWED_EXT = {'.png', '.jpg', '.jpeg', '.webp'}
MAX_UPLOAD_BYTES = int(os.environ.get("MAX_UPLOAD_BYTES", 6 * 1024 * 1024))
MAX_CONCURRENT_INFERENCES = int(os.environ.get("MAX_CONCURRENT", 1))
inference_semaphore = Semaphore(MAX_CONCURRENT_INFERENCES)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

STYLE_PRESETS = {
    "fantasy": {"add": "epic fantasy portrait, ornate details", "neg": "lowres, blurry"},
    "cyberpunk": {"add": "neon cyberpunk portrait", "neg": "lowres, watermark"},
    "anime": {"add": "anime portrait, clean lineart", "neg": "messy lines"},
    "oil-painting": {"add": "oil painting, chiaroscuro", "neg": "smudged"},
}

# Unique Name func
def gen_unique_name(length: int = 9) -> str:
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.choices(chars, k=length))

def batch_file_name(batch_id: str, file_id: str, role: str, ext: str) -> str:
    return f"{batch_id}_{file_id}_{role}{ext}"

app = Flask(__name__)
CORS(app)
app.app_context().push()

# GDrive
gdrive = None
try:
    if colab_auth is not None:
        try:
            colab_auth.authenticate_user()
        except Exception as e:
            print("[WARN] colab auth:", e)
    gauth = GoogleAuth()
    gdrive = GoogleDrive(gauth)
    print("[INFO] GoogleDrive client initialized.")
except Exception as e:
    print("[WARN] GoogleDrive not initialized:", e)
    gdrive = None

def upload_to_drive(local_path: str, parent_folder_id: str = None) -> Dict[str, str]:
    if gdrive is None:
        return {"error": "drive_not_initialized"}
    try:
        meta = {'title': os.path.basename(local_path)}
        if parent_folder_id:
            meta['parents'] = [{'id': parent_folder_id}]
        f = gdrive.CreateFile(meta)
        f.SetContentFile(local_path)
        f.Upload()
        try:
            f.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
        except Exception:
            pass
        f.FetchMetadata()
        return {
            "id": f.get('id'),
            "webViewLink": f.get('alternateLink'),
            "webContentLink": f.get('downloadUrl') or f.get('alternateLink')
        }
    except Exception as e:
        print("[ERROR] upload_to_drive:", e)
        return {"error": str(e)}

def ensure_batch_folder(batch_id: str, drive_client) -> str:
    if drive_client is None:
        return None
    try:
        safe_id = batch_id.replace("'", "\\'")
        q = f"title = '{safe_id}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        lst = drive_client.ListFile({'q': q}).GetList()
        if lst:
            return lst[0]['id']
        fld = drive_client.CreateFile({'title': batch_id, 'mimeType': 'application/vnd.google-apps.folder'})
        fld.Upload()
        return fld['id']
    except Exception as e:
        print("[ERROR] ensure_batch_folder:", e)
        return None

def allowed_file(fn: str) -> bool:
    _, ext = os.path.splitext(fn.lower())
    return ext in ALLOWED_EXT

def save_pil(img: Image.Image, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    img.save(path)

def pil_to_base64(img: Image.Image, fmt: str='PNG') -> str:
    buf = BytesIO()
    img.save(buf, format=fmt)
    buf.seek(0)
    return f"data:image/{fmt.lower()};base64,{base64.b64encode(buf.read()).decode('ascii')}"

# Load pipelines
pipe = None
text_pipe = None

use_fp16 = (DEVICE == "cuda") and torch.cuda.is_available()
torch_dtype = torch.float16 if use_fp16 else torch.float32

print(f"[INFO] Loading pipelines MODEL_ID={MODEL_ID} DEVICE={DEVICE} fp16={use_fp16}")

# load Img2Img
try:
    kwargs = {}
    if HF_TOKEN:
        kwargs['use_auth_token'] = HF_TOKEN
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(MODEL_ID, torch_dtype=torch_dtype, **kwargs)
    try:
        pipe = pipe.to(DEVICE)
    except Exception:
        pipe = pipe.to('cpu')
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception:
        pass
    print("[INFO] Img2Img pipeline loaded from", MODEL_ID)
except Exception as e:
    print("[WARN] Img2Img load failed for", MODEL_ID, ":", e)
    try:
        fallback = "runwayml/stable-diffusion-v1-5"
        print("[INFO] Attempting fallback Img2Img:", fallback)
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(fallback, torch_dtype=torch_dtype, **kwargs)
        pipe = pipe.to(DEVICE if torch.cuda.is_available() else 'cpu')
        print("[INFO] Fallback Img2Img loaded:", fallback)
    except Exception as e2:
        print("[ERROR] Fallback Img2Img load failed:", e2)
        pipe = None

# load text->image
try:
    kwargs = {}
    if HF_TOKEN:
        kwargs['use_auth_token'] = HF_TOKEN
    text_pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch_dtype, **kwargs)
    try:
        text_pipe = text_pipe.to(DEVICE)
    except Exception:
        text_pipe = text_pipe.to('cpu')
    try:
        text_pipe.enable_xformers_memory_efficient_attention()
    except Exception:
        pass
    print("[INFO] text2img pipeline loaded from", MODEL_ID)
except Exception as e:
    print("[WARN] text2img load failed for", MODEL_ID, ":", e)
    try:
        fb = "runwayml/stable-diffusion-v1-5"
        kwargs = {}
        if HF_TOKEN:
            kwargs['use_auth_token'] = HF_TOKEN
        text_pipe = StableDiffusionPipeline.from_pretrained(fb, torch_dtype=torch_dtype, **kwargs)
        text_pipe = text_pipe.to(DEVICE if torch.cuda.is_available() else 'cpu')
        print("[INFO] Fallback text2img loaded:", fb)
    except Exception as e2:
        print("[WARN] fallback text2img failed:", e2)
        text_pipe = None

def require_api_key(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        key = request.headers.get("X-API-KEY") or request.args.get("api_key")
        if not key or key != API_KEY:
            return jsonify({"error": "unauthorized"}), 401
        return f(*args, **kwargs)
    return wrapper

def safe_int(v, default=0):
    try:
        return int(v)
    except Exception:
        return default

# Endpoints

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        "ok": True,
        "img2img_loaded": pipe is not None,
        "text2img_loaded": text_pipe is not None,
        "device": DEVICE
    })

@app.route('/generate_text', methods=['POST'])
@require_api_key
def generate_text():
    if text_pipe is None:
        return jsonify({"error": "text_pipe_not_loaded"}), 500
    try:
        data = request.get_json(force=True)
    except Exception:
        return jsonify({"error": "invalid_json"}), 400

    prompt = (data.get("prompt") or "").strip()
    style = (data.get("style") or "").strip().lower()
    neg = (data.get("negative_prompt") or "").strip()
    if not prompt:
        return jsonify({"error": "missing_prompt"}), 400

    if style and style in STYLE_PRESETS:
        prompt += f", {STYLE_PRESETS[style]['add']}"
        if not neg:
            neg = STYLE_PRESETS[style].get('neg', '')

    steps = safe_int(data.get("num_inference_steps", 128), 328)
    guidance = float(data.get("guidance_scale", 7.5))
    batch_id = (data.get("batch_id") or gen_unique_name()).lower()
    file_id = gen_unique_name()
    ext = ".png"
    out_name = batch_file_name(batch_id, file_id, "out", ext)
    out_local = os.path.join(OUTPUT_DIR, out_name)

    acquired = inference_semaphore.acquire(timeout=300)
    if not acquired:
        return jsonify({"error": "inference_semaphore_timeout"}), 500
    try:
        out = text_pipe(prompt=prompt, negative_prompt=neg if neg else None,
                        num_inference_steps=steps, guidance_scale=guidance).images[0]
        save_pil(out, out_local)
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": f"generation_failed: {str(e)}"}), 500
    finally:
        inference_semaphore.release()

    drive_links = {}
    if gdrive is not None:
        fid = ensure_batch_folder(batch_id, gdrive)
        if fid:
            drive_links = upload_to_drive(out_local, fid)

    return jsonify({
        "batch_id": batch_id,
        "file_id": file_id,
        "base64": pil_to_base64(out),
        "output_local": out_local,
        "drive": drive_links
    }), 200

@app.route('/generate_from_upload', methods=['POST'])
@require_api_key
def generate_from_upload():
    if pipe is None:
        return jsonify({"error": "img2img_pipeline_not_loaded"}), 500

    file = request.files.get('file')
    if not file:
        return jsonify({"error": "no_file"}), 400
    filename = secure_filename(file.filename or "upload.png")
    if not allowed_file(filename):
        return jsonify({"error": "invalid_file_type"}), 400

    batch_id = (request.form.get("batch_id") or gen_unique_name()).lower()
    file_id = gen_unique_name()

    style = (request.form.get("style") or "").strip().lower()
    prompt = (request.form.get("prompt") or "").strip()
    if style and style in STYLE_PRESETS:
        prompt += f", {STYLE_PRESETS[style]['add']}"
    neg = (request.form.get("negative_prompt") or "").strip()
    if not neg and style and style in STYLE_PRESETS:
        neg = STYLE_PRESETS[style].get('neg', "")

    try:
        # size check
        file.stream.seek(0, 2)
        filesize = file.stream.tell()
        file.stream.seek(0)
    except Exception:
        data = file.read()
        filesize = len(data)
        file.stream = BytesIO(data)
    if filesize > MAX_UPLOAD_BYTES:
        return jsonify({"error": "file_too_large"}), 400

    ext = os.path.splitext(filename)[1].lower()
    local_in = os.path.join(INPUT_DIR, batch_file_name(batch_id, file_id, "in", ext))
    local_out = os.path.join(OUTPUT_DIR, batch_file_name(batch_id, file_id, "out", ext))

    try:
        img = Image.open(file.stream).convert("RGB")
        save_pil(img, local_in)
    except Exception as e:
        return jsonify({"error": f"cannot_open_uploaded_image: {e}"}), 400

    strength = float(request.form.get("strength", 0.7))
    steps = safe_int(request.form.get("num_inference_steps", 342), 342)
    guidance = float(request.form.get("guidance_scale", 8.5))

    acquired = inference_semaphore.acquire(timeout=300)
    if not acquired:
        return jsonify({"error": "inference_semaphore_timeout"}), 500

    try:
        out_img = pipe(
            prompt=prompt,
            image=img,
            negative_prompt=neg if neg else None,
            strength=strength,
            guidance_scale=guidance,
            num_inference_steps=steps,
        ).images[0]
        save_pil(out_img, local_out)
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": f"img2img_failed: {str(e)}"}), 500
    finally:
        inference_semaphore.release()

    drive_in = {}
    drive_out = {}
    if gdrive is not None:
        folder_id = ensure_batch_folder(batch_id, gdrive)
        if folder_id:
            drive_in = upload_to_drive(local_in, folder_id)
            drive_out = upload_to_drive(local_out, folder_id)

    return jsonify({
        "batch_id": batch_id,
        "file_id": file_id,
        "input_local": local_in,
        "output_local": local_out,
        "input_drive": drive_in,
        "output_drive": drive_out,
        "base64": pil_to_base64(out_img)
    }), 200

@app.route('/upload_for_training', methods=['POST'])
@require_api_key
def upload_for_training():
    file = request.files.get('file')
    if not file:
        return jsonify({"error": "no_file"}), 400
    filename = secure_filename(file.filename or "train.png")
    if not allowed_file(filename):
        return jsonify({"error": "invalid_file_type"}), 400

    try:
        file.stream.seek(0, 2)
        filesize = file.stream.tell()
        file.stream.seek(0)
    except Exception:
        data = file.read()
        filesize = len(data)
        file.stream = BytesIO(data)

    if filesize > (10 * 1024 * 1024):
        return jsonify({"error": "file_too_large"}), 400

    label = (request.form.get("label") or "unlabeled").strip()
    folder = os.path.join(META_DIR, "training_samples", secure_filename(label))
    os.makedirs(folder, exist_ok=True)
    ts = int(time.time())
    unique = gen_unique_name()
    local = os.path.join(folder, f"{ts}_{unique}_{filename}")
    try:
        img = Image.open(file.stream).convert("RGB")
        save_pil(img, local)
        return jsonify({"status": "ok", "local_path": local}), 200
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": f"save_failed: {str(e)}"}), 500

# Minimum Frontend
HTML = """
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<title>RWC-Minus-Studio</title>
<meta name="viewport" content="width=device-width,initial-scale=1">
<style>
  body {
    font-family: "Share Tech Mono", monospace, system-ui;
    background: #0a0a0f;
    margin: 20px;
    color: #e0e0e0;
  }
  h2 {
    color: #00ffe7;
    text-shadow: 0 0 6px #00ffe7, 0 0 12px #008080;
    letter-spacing: 1px;
  }
  label {
    display: block;
    margin-top: 10px;
    color: #a0f0ff;
    font-size: 14px;
  }
  input, button, select, textarea {
    font-family: inherit;
    font-size: 14px;
    padding: 8px;
    border-radius: 6px;
    border: 1px solid #333;
    background: #111;
    color: #eee;
    outline: none;
    margin-top: 4px;
  }
  select, textarea {
    width: 100%;
  }
  button {
    cursor: pointer;
    background: linear-gradient(90deg, #00ffe7, #0066ff);
    border: none;
    color: black;
    font-weight: bold;
    transition: all 0.2s ease;
  }
  button:hover {
    transform: scale(1.05);
    box-shadow: 0 0 12px #00ffe7;
  }
  .preview img {
    max-width: 640px;
    border: 2px solid #00ffe7;
    border-radius: 10px;
    margin-top: 12px;
    box-shadow: 0 0 10px #00ffe7;
  }
  .muted {
    color: #888;
    font-size: 13px;
    margin-top: 8px;
  }
</style>
<link href="https://fonts.googleapis.com/css2?family=Share+Tech+Mono&display=swap" rel="stylesheet">
</head>
<body>
<h2>RWC-Minus-Studio</h2>
<div>
  <label>Mode:
    <select id="mode">
      <option value="text">Text → Image</option>
      <option value="edit">Image → Image (Edit)</option>
      <option value="enhance">Enhance</option>
    </select>
  </label>

  <label>Image (for edit/enhance / training):
    <input id="file" type="file" accept="image/*">
  </label>

  <label>Prompt:</label>
  <textarea id="prompt" rows="3" placeholder="Enter prompt (for text→image or edit)"></textarea>

  <label>Style:
    <select id="style">
      <option value="">(none)</option>
      <option value="fantasy">fantasy</option>
      <option value="cyberpunk">cyberpunk</option>
      <option value="anime">anime</option>
      <option value="oil-painting">oil-painting</option>
    </select>
  </label>

  <div style="margin-top:12px;">
    <button onclick="generate()">Generate</button>
    <button onclick="uploadTraining()">Upload for training</button>
  </div>
</div>

<div id="status" class="muted"></div>
<div class="preview" id="preview"></div>

<script>
const API_KEY = 'changeme_local_only'; // change before deploy

async function generate(){
  const mode = document.getElementById('mode').value;
  const fileInput = document.getElementById('file');
  const prompt = document.getElementById('prompt').value.trim();
  const style = document.getElementById('style').value;
  const status = document.getElementById('status');
  const preview = document.getElementById('preview');
  status.textContent = '⚡ Processing...';
  preview.innerHTML = '';
  try{
    if(mode === 'text'){
      const res = await fetch('/generate_text', {
        method:'POST',
        headers:{'Content-Type':'application/json','X-API-KEY':API_KEY},
        body: JSON.stringify({prompt, style})
      });
      const data = await res.json();
      if(!res.ok){ status.textContent = '❌ Failed: ' + (data.error||res.statusText); return; }
      preview.innerHTML = '<div><img src="'+data.base64+'"></div>';
      status.textContent = '✅ Done (text→image) | batch:'+data.batch_id+' id:'+data.file_id;
    } else {
      if(!fileInput.files || !fileInput.files[0]){ status.textContent='⚠️ Select an image for edit/enhance'; return; }
      const fd = new FormData();
      fd.append('file', fileInput.files[0]);
      fd.append('prompt', prompt);
      fd.append('mode', mode);
      fd.append('style', style);
      const res = await fetch('/generate_from_upload', {method:'POST', headers:{'X-API-KEY':API_KEY}, body: fd});
      const data = await res.json();
      if(!res.ok){ status.textContent = '❌ Failed: ' + (data.error||res.statusText); return; }
      preview.innerHTML = '<div><img src="'+data.base64+'"></div>';
      status.textContent = '✅ Done (image→image) | batch:'+data.batch_id+' id:'+data.file_id;
    }
  } catch(e){
    console.error(e);
    status.textContent = '🔥 Error: ' + e.message;
  }
}

async function uploadTraining(){
  const fileInput = document.getElementById('file');
  const status = document.getElementById('status');
  if(!fileInput.files || !fileInput.files[0]){ status.textContent='⚠️ Select an image to upload for training'; return; }
  const fd = new FormData();
  fd.append('file', fileInput.files[0]);
  fd.append('label','user_upload');
  status.textContent = '⏫ Uploading for training...';
  try{
    const res = await fetch('/upload_for_training', {method:'POST', headers:{'X-API-KEY':API_KEY}, body:fd});
    const data = await res.json();
    if(!res.ok){ status.textContent = '❌ Upload failed: '+(data.error||res.statusText); return; }
    status.textContent = '✅ Uploaded for training: ' + data.local_path;
  } catch(e){
    console.error(e);
    status.textContent = '🔥 Error: ' + e.message;
  }
}
</script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML)

if __name__ == '__main__':
    try:
        if NGROK_DOMAIN:
            try:
                os.system(f"ngrok http 80 --domain={NGROK_DOMAIN} &")
                print("[INFO] started ngrok with domain:", NGROK_DOMAIN)
            except Exception as e:
                print("[WARN] ngrok start failed:", e)
        print("[INFO] Starting Flask on 0.0.0.0:80")
        app.run(host='0.0.0.0', port=80)
    finally:
        pass